<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**<font size=6>🌌Stellar</font>**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json file is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 27jan25-stellar           # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
lb =!kaggle competitions leaderboard --show                   # print public leaderboard

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
- competition is now set to: 27jan25-stellar
100% 7.58M/7.58M [00:00<00:00, 61.4MB/s]


In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 2.65 s, sys: 309 ms, total: 2.96 s
Wall time: 4.46 s


In [ ]:
df = pd.read_csv('XY_Stellar.csv'); df

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199998,131.31,44.27,24.07,24.64,21.63,19.20,19.03,7076,3,251,0.55,6014,56166,1021,G
199999,22.59,0.25,25.30,25.56,24.09,19.41,19.96,5164,4,511,1.26,9590,57969,878,G


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [ ]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)

In [ ]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample. Class!=Class is True for NaN values
tXY = df.query('Class==Class')                       # slice a train sample. Class==Class is True for non-NaN values
tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [ ]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))  # may take a few minutes to run

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green>**Import Necessary Libraries**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

np.random.seed(0)

<font color=green>**Stratified Sampling of the Dataset**

In [ ]:
from sklearn.model_selection import train_test_split

sample_size = 25000

# Stratified sampling to preserve class distribution
tX, _, tY, _ = train_test_split(
    tX, tY, train_size=sample_size, stratify=tY, random_state=42
)

<font color=green>**Feature Selection**

In [ ]:
tX = tX[['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']]
vX = vX[['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']]

<font color=green>**Feature Engineering**

In [ ]:
redshift_shift = abs(min(tX['redshift'].min(), vX['redshift'].min())) if min(tX['redshift'].min(), vX['redshift'].min()) < 0 else 0
tX['sqrt_redshift'] = np.sqrt(tX['redshift'] + redshift_shift + 1e-6)  # Small epsilon to avoid zero issues
vX['sqrt_redshift'] = np.sqrt(vX['redshift'] + redshift_shift + 1e-6)

tX['ug_ratio'] = tX['u'] / np.where(tX['g'] == 0, 1e-6, tX['g'])
tX['gr_ratio'] = tX['g'] / np.where(tX['r'] == 0, 1e-6, tX['r'])
tX['ri_ratio'] = tX['r'] / np.where(tX['i'] == 0, 1e-6, tX['i'])
tX['iz_ratio'] = tX['i'] / np.where(tX['z'] == 0, 1e-6, tX['z'])

vX['ug_ratio'] = vX['u'] / np.where(vX['g'] == 0, 1e-6, vX['g'])
vX['gr_ratio'] = vX['g'] / np.where(vX['r'] == 0, 1e-6, vX['r'])
vX['ri_ratio'] = vX['r'] / np.where(vX['i'] == 0, 1e-6, vX['i'])
vX['iz_ratio'] = vX['i'] / np.where(vX['z'] == 0, 1e-6, vX['z'])

<ipython-input-21-8e61cc3fcafb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vX['sqrt_redshift'] = np.sqrt(vX['redshift'] + redshift_shift + 1e-6)
<ipython-input-21-8e61cc3fcafb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vX['ug_ratio'] = vX['u'] / np.where(vX['g'] == 0, 1e-6, vX['g'])
<ipython-input-21-8e61cc3fcafb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

<font color=green>**Polynomial Features and Scaling**

In [ ]:
from sklearn.pipeline import Pipeline

#Create model Pipeline
m = Pipeline([
    ('poly', PolynomialFeatures(degree=3)),
    ('standardize', StandardScaler()),
    ('clr', LR(max_iter=600, C=10, random_state=0))
])

<font color=green>**Fit the Logistic Regression Model to the Training Set**


In [ ]:
m.fit(tX, tY)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('poly', PolynomialFeatures(degree=3)),
                ('standardize', StandardScaler()),
                ('clr',
                 LogisticRegression(C=10, max_iter=600, random_state=0))])

<font color=green>**Training Accuracy**

In [ ]:
#training_accuracy
print(m.score(tX, tY))

0.96836


<font color=green>**Make Prediction for the Testing Set**

In [ ]:
#Predict Requested Values
prediction = m.predict(vX)


pY = pd.DataFrame(prediction, index=range(1,len(vX)+1), columns=['Class'])  # ensure that labels and observations are in corresponding order
pY.value_counts()
ToCSV(pY, 'attempt')

,count
Class,
G,23987
S,8909
Q,7104


<font color=green><h3><b>$\beta$. Idea Documentation</b></h3>
<details>
  <summary>Instructions</summary>
  <div>


1. **Audience**. Your peers who will learn from your Colab and ideas therein.
1. **Importance**. The ML/DL ideas are not entirely random, but are based on prior experience and systematized/organized experiments. We'd like students to share and learn from idea generation to idea experimentation process done in our class using tools learned thus far.
1. **Format**. Keep it concise/precise in consistent font/presentation. Include numbers/IDs to your References, such as [1] or [[Géron22]](https://scholar.google.com/scholar?cluster=498861685923226475), where these are defined in your References section below. This helps link your ideas/experiments to external ideas.
1. **Reproducibility**. Your description should contain reasonable details needed for reproducibility, i.e. describe the state of your modeling pipeline before the change is made, what is changed and how the idea was discovered, and what improvement it resulted in. Thus, peers can try this idea with an expectation of the value it brings. See examples below.
1. **Bonus** points for the exceptional/exemplary/educational documentation (see grading rubric).
****
1. **TODO**: Describe the key idea in your work in the following format (similar to a "micro publication"):
  1. **Title**. Give each idea a descriptive name (i.e. a micro abstract).
    1. Ex(ample). <i>"Thresholding carat feature outliers improves MAE by 3% on public LB"</i>
  1. **Idea Discovery**. What led you to this idea? Was it some [EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis), familiarity with this dataset or some of the features?
    1. Ex. <i>"We plotted all univariate distributions of variables and discovered that diamond carat had unreasonable (but rare) values below and above [0,10] interval, when plotted carat's histogram in the train and test sets, which contained 10 and 3 such outliers respectively. We decided to use 10 as a reasonable threshold because it is 99th percentile of carat values in the 20K baseline sample. See our histogram plot below [plot here]. "</i>
  1. **Finding's Importance**. Describe why you think the idea was important to proceed with.
    1. Ex. <i>"We use a linear model, the slope of which is sensitive to outliers on the periphery of the feature space domain. The fitted hyperplane slopes in the direction of the extreme training feature values thereby mapping a non-existent relation between carat size and diamond price, which is not expected to repeat in the test set. "</i>
  1. **Experiment Setup**.
  How did you set up experiments to test your idea? What resources were helpful? What metric did you select, why and what values did you observe?
    1. Ex. <i>"To alleviate the impact of the outlying feature values, we need to either remove observations with extreme values, or somehow cap them (to stay within the distribution of the other carat values) or use a model insensitive to outliers (such as robust regression). We learned 3 suitable methods for treating outliers in [ref]: ... [It'd be great to briefly describe each method] We tried each one on a Baseline model, while keeping the competition-required [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) metric. We tested each method locally on the seeded 50/50 split of the 20K training set sampled in baseline Colab."</i>
  1. **Results**. What was the result or metric improvement from implementing the experiment locally and/or on public LB?
    1. Ex. <i>"Baseline MAE was 539.1257546465 in public LB and 530 in local default experiment with 50/50 train-test split. When applied on the same-seed split, Methods 1,2,and 3 showed 1%, 2%, and 5% improvement on the test set. When uploaded to public LB, Method 3 showed a 3% improvement. So, we decided to keep method 3."</i>

</div> </details>
</font>


<font color=green><h4><b>Task 1. Preprocessing Ideas</b></h4>
<details>
  <summary>Instructions</summary>
  <div>Explain a <b>key idea</b> that helped in <b>preprocessing pipeline</b>. This may be about some feature engineering, tricky subsampling, clustering, dimension reduction, etc. Use the format in TODO specified above. Remember to provide citation references for the peers to read more into your work.
</div> </details>
</font>

1. **Title**: Removal of Unrelated Features
1. **Idea Discovery**: During exploratory data analysis, we noticed several ID-based features (run_id, cam_col, plate, MJD, fiber_ID) that did not contain meaningful information for classification. As per the dataset provided, the run_ID is identification of the specific scan, the field_ID is the identification of the field, fiber_ID is the identification of the fiber that points the light at the focal plane, the plate ID is an identification of each plate is SDSS, and the MJD, the Modified Julian Date, refer to when the data was taken [1]. We examined alpha (Right Ascension) and delta (Declination), which represent celestial coordinates in the J2000 epoch [2]. While these features describe an object's position in the sky, their relevance to spectral classification was uncertain. Given that classification is based on spectral properties rather than position, we hypothesized that alpha and delta might be redundant.
1. **Finding's Importance**: Features that do not contribute to learning decision boundaries can add unnecessary complexity, increase computation time and potentially introduce noise. Their removal allows the model to focus on relevant variables.
1. **Experiment Setup**: We used five-fold cross-validation with the Logistic Regression model as a baseline with all features and another model after removing ID-bases features. The same pre-processing steps and hyperparameters were applied to both models to ensure comparability. We also conducted an incremental feature removal approach to confirm the relevance of each feature.
1. **Results**: Classification accuracy increases from 90% to 95% after removing ID-based features, confirming their irrelevance in classification. Retaining alpha and delta further boosted accuracy, suggesting that positional information contributes some predictive value, potentially due to spatial correlations in the dataset.

1. **Title**: Log/Sqrt Transformations
1. **Idea Discovery**: During exploratory data analysis, we observed that several features in the dataset exhibited skewed distributions and large variations in scale. These included alpha, delta, redshift, and the photometric magnitudes - u, g, r, i, z. Given the importance of numerical stability in machine learning models, we investigated whether applying logarithmic and square root transformations would enhance linear separability and improve classification performance. Additionally negative values in some features required shifting before applying transformations.
1. **Finding's Importance**: Skewed and high-magnitude numerical features can distort relationships within the data, making it difficult for models to generalize well. Logarithmic and square root transformations help stabilize variance, normalize distributions and reduce the impact of extreme values, leading to feature relationships. As outliers were not removed (see section on outliers above), data transformations are pertinent to maintain the integrity of the data. If a transformation results in a more linearly separable feature space, classification algorithms such a Logistic Regression, LDA and QDA can differentiate between classes.
1. **Experiment Setup**: We applied both log and square root functions to the relevant features and assessed their impact using five-fold cross validation.
1. **Results**: Overall, only the square root of redshift resulted in visibly clear linear separation between the three classes. The other transformations, while still lending towards a more normal distribution, resulted in a decrease or negligible change in performance.

1. **Title**: Feature Engineering
1. **Idea Discovery**: Astronomical classification relies heavily on photometric data, specifically how objects emit light across different wavelengths. Color indices and color ratios are well-established in astrophysics as they help distinguish between starz, galaxies and quasars by capturing spectral characteristics [3, 4, 5]. Given that the dataset includes magnitudes in five filters (u, g, r, i, z), we explore feature engineering techniques to enhance classification performance. In addition to standard first-order color indices, we experimented with color ratios and second-order spectral features to assess their effectiveness in separating different celestial objects.
1. **Finding's Importance**: Raw magnitude values (u, g, r, i, z) do not provide relative brightness comparisons across filters. Instead differences and ratios between these values offer more meaningful spectral trends, which can be crucial for distinguishing object types.
  1. First-Order Color Indices: help differentiate celestial objects by measuring brightness differences between bands [u - g, g - r, r - i, i - z] [3, 4]
  2. Color Ratios: capture relationships between magnitudes in adjacent bands, helping normalize spectral variations. [ u/g, g/r, r/i, i/z] [4, 5]
  3. Second-Order Color Indices were introduced to capture curvature trends in an object’s spectral energy distribution. [ ((u-g) - (g - r)), ((g-r) - (r - i)), ((r - i) - (i - z))
1. **Experiment Setup**: We tested various feature transformations using five-fold cross-validation on a Logistic Regression model, with polynomial features (degree = 3) and StandardScaler. We incrementally added the new features to the dataset, and evaluated the change in performance of the classification model. Additionally, we experimented with applying logarithmic and square root transformations on the new features.
1. **Results**: It was found that the first-order color indices and the color ratios, without log and square root transformations gave the strongest results. Only one of the second-order color indices, ((u-g) - (g - r)), proved to have minimal predictive value; however, the slight improvement in accuracy was outweighed by the noise generated by the feature. Ultimately, the first-order color indices and the color ratios were maintained in the final model.

1. **Title**: Evaluating Outliers using z-score
1. **Idea Discovery**: Since the given dataset contains only numerical features as opposed to categorical features, it has been contemplated that values outside a certain degree of quantiles could exist and impact on model performance. We explored outlier detection using z-scores.
1. **Finding's Importance**: The existence of outliers in the dataset can significantly skew summary statistics such as the mean and standard deviation. As preprocessing techniques like standardization rely on these statistics, the feature scaling becomes less effective, and the model could potentially not learn the true relationships between features and the target variable, introducing noise, leading to poor generalization [6].
1. **Experiment Setup**: We calculated z-scores for each feature and identified outliers as values beyond a threshold. These values were removed from the dataset. A baseline five-fold cross validation model was trained, followed by another model with outlier removal to compare the results.
1. **Results**: Outliers accounted for a small fraction of the dataset (~0.018 in the most extreme case). Removing them did not lead to improvements in model performance, suggesting that the outliers were not significantly skewing the data.

<font color=green><h4><b>Task 2. Modeling Ideas</b></h4>
<details>
  <summary>Instructions</summary>
  <div>Explain a <b>key idea</b> that helped with <b>model selection</b> in the format specified above. This may include tuning model parameters (perhaps a grid search with specific parameter range) or some other experiments, search/choice of the suitable model, experiments with postprocessing of model predictions, etc. Use the format in TODO specified above. Remember to provide citation references for the peers to read more into your work.
</div> </details>
</font>

1. **Title**: Standard Scaler vs. Robust Scaler vs. MinMaxScaler
1. **Idea Discovery**: During exploratory data analysis, it was discovered that the range of which features spread out varies significantly. Due to this, we decided to explore scaling to minimize the inconsistent spread of data. Given the presence of outliers, we considered whether a RobustScaler (which is more resistant to outliers) would outperform StandardScaler and MinMaxScaler.
1. **Finding's Importance**: Scaling data is important because it allows the data to avoid dominance by large scale features, and enhances the model with a regularization effect. When using regularization, features on vastly different scales can result in an uneven application of the regularization term. Scaling ensures that the penalty is applied uniformly across all features, eventually leading to improving the model’s ability to generalize. With the general scaling purpose, whether handling outliers with a specific type of scaler is important in order to alleviate degrading effects from outliers [7].
1. **Experiment Setup**: A five-fold cross validation model was trained without any scaling as a baseline. This was repeated using MinMaxScaler, RobustScaler and StandardScaler, comparing their impact on model performance.
1. **Results**: On the cross-validation set, the StandardScaler performed with higher accuracy, compared to the RobustScaler, suggesting that the RobustScaler discarded outliers that may have been pertinent to the dataset. We chose to proceed with the StandardScaler as it retained the integrity of the dataset.

1. **Title**: Normalization
1. **Idea Discovery**: Many of the data fields had a very wide range of numerical values. Due to this we decided to attempt to see if normalization of these values would help the model pick up on patterns better.
1. **Finding's Importance**: Categories with a wide range of values can cause the model to struggle with interpreting relationships. By standardizing gesture ranges, normalization allows the model to treat all features equitably, thus improving its ability to detect and leverage underlying patterns.
1. **Experiment Setup**: Utilizing a few different Normalization methods including: Min-Max Scaling, Z-score Standardization, and Log Transformations models were trained with no other changes. This allowed us to calculate an accuracy score to determine if any form of Normalization aided in the model's accuracy.
1. **Results**:  No model performed better with the inclusion of any normalization method.

1. **Title**: Fine-tuning Polynomial Features
1. **Idea Discovery**: Feature engineering involves generating polynomial features to capture nonlinear relationships. We explored different polynomial degrees to find the optimal balance between feature expressiveness and overfitting risk.
1. **Finding's Importance**: Higher-degree polynomial features can enhance separation between classes, but excessive complexity may introduce noise. Additionally, introducing complexity runs the risk of causing overfitting the model to the training dataset.
1. **Experiment Setup**: Using five-fold cross validation, we tested polynomial degrees of 2, 3, 4 and 5. In addition to comparing the validation accuracy, we also assessed the training accuracy to monitor overfitting.
1. **Results**: Degree 3 was found to be the optimal choice. Compared to degree 2, degree 3 increased testing accuracy from 91% to 93%, suggesting that polynomial features of degree 3 were able to capture the non-linear relationships of the dataset. Degree 4 showed a slightly higher training accuracy than degree 3; however, this same accuracy increase was not reflected in the validation accuracy, suggesting there was overfitting in the dataset.

1. **Title**: Dimensionality Reduction
1. **Idea Discovery**: Given the high number of features introduces thorough polynomial transformation (degree = 3) and engineered spectral features, we explored Principal Component Analysis (PCA) and Linear Discriminant Analysis (LDA) to reduce dimensionality while preserving critical classification information.
1. **Finding's Importance**: Dimensionality reduction can help mitigate the curse of dimensionality, improve model generalization, and reduce computational complexity. PCA is effective in capturing variance, while LDA optimizes for class separability, making it particularly relevant for classification tasks.
1. **Experiment Setup**: We applied PCA (10 components) and LDA (max components = 2 [max_classes - 1]) to transformed features and evaluated their impact using five-fold cross validation. Both PCA and LDA were tested alongside Logistic Regression with polynomial features (degree = 3) and StandardScaler.
1. **Results**: LDA outperformed PCA, achieving a validation accuracy of 95.6%, compared to 81.8%. However, the baseline model without any dimensionality reduction achieved the best performance of 96.8%. The likely reason for this is that polynomial features capture complex relationships within the dataset that emphasize the separation of the three classes.

1. **Title**: GridSearchCV to tune Hyperparameters
1. **Idea Discovery**: While working on model training, we wanted to explore how different combinations of hyperparameters would affect model performance. Given that hyperparameters, such as the regularization strength and preprocessing methods, play a significant role in model accuracy, we decided to use GridSearchCV to systematically search for the best combination. This approach would help us optimize the model’s configuration to improve performance, as we noticed performance variability when changing hyperparameters manually.
1. **Finding's Importance**: Hyperparameter tuning is crucial in machine learning as it can significantly impact the model's ability to generalize. By using GridSearchCV, we were able to automate the process of testing various hyperparameter combinations, ensuring we identified the optimal setup.
1. **Experiment Setup**:
  1. We determined a range of potential values for each hyperparameter:
    1. Scalar: [Standard Scaler, MinMax Scaler, Robust Scaler]
    2. Polynomial Features Degree: [1, 2, 3, 4]
    3. Models: [LogisticRegression, LinearSVC, LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis, Ridge, RidgeCV]
    4. Regularization Strength: [0.01, 0.1, 1, 5, 10]
    5. Penalty: [‘l1’, 'l2']
    6. Model solvers: ['svd', 'lsqr', 'eigen']
  2. We then ran GridSearchCV to determine the best combinations of these parameters utilizing Accuracy as our scorer.
1. **Results**: While SVC tended to come out on top with, in practice it ran too slow to be utilized for this project. The second place was LogisticRegression which ran much quicker utilizing PolynomialDegree=3, StandardScaler, and C=10.

<font color=green><h3><b>$\gamma$. References</b></h3>
<details>
  <summary>Instructions</summary>
  <div>

1. Cite your sources to help your peers learn from these (and to avoid plagiarism).
1. HOML textbook should be cited, since we used it in this week's learning.
1. Use Google Scholar to draw [APA](https://en.wikipedia.org/wiki/American_Psychological_Association) citation format for books and publications.
1. Cite [StackOverflow](https://stackoverflow.com/), YouTube videos, package docs, open-access textbooks/publicaitons and other meaningful internet resources that you used.
1. We may reward exceptional and meaningful citations (not just a list of [SKL](https://scikit-learn.org/stable/)/[TF](https://www.tensorflow.org/) manual pages and a list of articles.) For example, if you used an idea from a publication, indicate it in TGP with a number that corresponds to its reference in References.

</div> </details>
</font>

1. Pan, D. C., Vogeley, M. S., Hoyle, F., Choi, Y.-Y., & Park, C. (2012). Cosmic voids in Sloan Digital Sky Survey Data Release 7. *Monthly Notices of the Royal Astronomical Society*, 421(2), 926–934. https://doi.org/10.1111/j.1365-2966.2011.20197.x
1. King, B. (2023, June 2). *Right Ascension & Declination: Celestial coordinates for beginners*. Sky & Telescope. https://skyandtelescope.org/astronomy-resources/right-ascension-declination-celestial-coordinates/
1. Fukugita, M., Ichikawa, T., Gunn, J. E., Doi, M., Shimasaku, K., & Schneider, D. P. (1996). The sloan digital sky survey photometric system. *The Astronomical Journal, 111*, 1748. https://doi.org/10.1086/117915
1. *Ugriz - (Intro to Astronomy)*. Fiveable. (n.d.). https://library.fiveable.me/key-terms/intro-astronomy/ugriz
1. Bessell, M. S. (2005). Standard Photometric Systems. *Annual Review of Astronomy and Astrophysics*, 43(1), 293–336. https://doi.org/10.1146/annurev.astro.41.082801.100251
1. GeeksforGeeks. (n.d.). *Z-Score for outlier detection in Python.* Retrieved February 8, 2025, from https://www.geeksforgeeks.org/z-score-for-outlier-detection-python/
1. GeeksforGeeks. (n.d.). *StandardScaler, MinMaxScaler and RobustScaler Techniques in ML*. Retrieved February 8, 2025, from  https://www.geeksforgeeks.org/standardscaler-minmaxscaler-and-robustscaler-techniques-ml

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 48 sec


<details>
  <summary><font size=5><b>💡Starter Ideas</b></font></summary>
  <div>
  
1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features?
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)

</div> </details>